# Build data of price and volume from Binance

Quick blowse [binance python-api docs](https://python-binance.readthedocs.io/en/latest/index.html) and get a rough profile of what we can do. Although it is an unofficial Python wrapper, but it is easy to use for trials. 

## Install Binance API
First, let's install Binance python module use pip command.

In [2]:
pip install binance

Note: you may need to restart the kernel to use updated packages.


In [2]:
import binance
binance.__file__

'/Users/Hans/.conda/envs/orange3/lib/python3.7/site-packages/binance.py'

In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


## Apply key and secret from Binance
Secord, apply an api key and secret from your Binance account.

In [1]:
from binance.client import Client
api_key = '7RS8At6SToQzDYTJpMT3yPOalVPkMZzeX6MnT2gmD6YzH0gjNHPLcvgmHXaHu00G'
api_secret = 'bJIz0uS5GwdiqrvijKfpsBGaR7NJJ8MpiRPNzPmrFB9W3E6arL4BqxLkLstYBPaa'
client = Client(api_key, api_secret, {'timeout':20})

ModuleNotFoundError: No module named 'binance.client'; 'binance' is not a package

## Request instruments and exchange info you need

In [ ]:
from binance.websockets import BinanceSocketManager
from binance.enums import *
import time 
import numpy as np
import pandas as pd
import sys, os

def product(client, num=50, market='USDT'):
    products = client.get_products()
    df = pd.DataFrame(products['data'])
    df = df[df['market'] == market]
    df['symbol'] = df['baseAsset']+df['quoteAsset']
    product_list = df.sort_values("tradedMoney", ascending=False).symbol.values[:num]
    return product_list

product_list = product(client, 200, 'USDT')
product_list

## Load history price and volume data
Let's make some utility functions for dates format convert. 

In [ ]:
def UTC_convert(x):
    if isinstance(x, np.ndarray):
        return np.array([time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(i)) for i in x/1000.])
    elif isinstance(x, pd.Series):
        return np.array([time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(i)) for i in x.values/1000.])
    elif (isinstance(x, int)) and (len(str(x)) ==13):
        return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x/1000.))
    else:
        raise Exception('error %s' %type(x))
        
def load_history_kline(history_kline):
    label = ["Open time", "Open", "High", "Low", "Close", "Volume", "Close time", "Quote asset volume", "Number of trades", "Taker buy base asset volume", "Taker buy quote asset volume", "Can be ignored"]
    kline_df = pd.DataFrame(history_kline, columns=label).astype(np.float)
    kline_df['Open time'] = UTC_convert(kline_df['Open time'])
    kline_df.index = kline_df['Open time'].values
    kline_df['Close time'] = UTC_convert(kline_df['Close time'])
    return kline_df

data = {}
datalen = 600
from tqdm import tqdm
data['OpenPrice'], data['ClosePrice'], data['HighestPrice'], data['LowestPrice'], data['Volume'], data['dates'], data['TradeStatus'] = {}, {}, {}, {}, {}, {}, {}
for i in tqdm(product_list):
    try:
        klines = client.get_historical_klines(i, Client.KLINE_INTERVAL_1DAY, "1 Jan, 2018")
    except:
        pass
    klines_df = load_history_kline(klines)
    print(i, klines_df.shape[0])
    if klines_df.shape[0] > datalen:
        data['OpenPrice'][i] = klines_df['Open'].values[-datalen:]
        data['ClosePrice'][i] = klines_df['Close'].values[-datalen:]
        data['HighestPrice'][i] = klines_df['High'].values[-datalen:]
        data['LowestPrice'][i] = klines_df['Low'].values[-datalen:]
        data['Volume'][i] = klines_df['Volume'].values[-datalen:]
        data['dates'][i] = klines_df['Open time'].values[-datalen:]
        data['TradeStatus'][i] = klines_df['Volume'].values[-datalen:]>0

## Store array-like data in local path

In [ ]:
from tqdm import tqdm
def store_data(data, path):
    for i in data.keys():
        file = os.path.join(path, i+'.npy')
        arr = pd.DataFrame(data[i]).T.values
        if i == 'dates':
            print(arr)
            arr = arr[0].astype('str')
        print(file, arr.shape)
        np.save(file, arr)
    t_path = os.path.join(path, 'ticker_names.npy')
    ticker_names = pd.DataFrame(data[i]).columns.values.astype('str')
    #print(ticker_names)
    np.save(t_path, ticker_names)
    print(t_path, ticker_names.shape)
    
store_data(data, path='/Users/Hans/Desktop/HunterAlpha/KitKitDataSets/BinanceData/1day/')

In [ ]:
def amount_convert(amount, precision):  
    """
    amount = 0.000234234
    precision = 5
    """
    amt_str = "{:0.0{}f}".format(amount, precision)
    return amt_str

